<a href="https://colab.research.google.com/github/superspray/KOR_DA_2021/blob/main/KOR_DA_2021_EDA_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 형태소 분석 기반의 쉬운 데이터 증강 (EDA-POS)



*   Wei and Zou (2019)의 방법을 한국어 데이터 특성에 맞게 변형한
형태소 분석 기반의 쉬운 데이터 증강 기법

*   형태소 분석기를 통한 품사 태깅 후 조사나 문장부호에 해당하는 것으로 분석된 단어들을 기법 적용대상에서 제외하고, 나머지 단어들에 대해 아래의 네가지 방법을 적용함


1.   동의어 대체 (synonym replacement, SR): 문장 내 임의로 추출된 p × l개의 단어들을 각각의 동의어 또는 유의어로 대체 (단, l 은
문장의 단어 수를 의미한다.)
2.   동의어 임의 삽입 (random Insertion, RI): 임의로 선택된 p × l개의
단어에 대한 동의어 또는 유의어를 문장 내 임의의 위치에 삽입 (단,
l은 문장의 단어 수를 의미한다.)
3.    임의 교환 (random Swap, RS): 임의로 두 개 단어를 선택하여 이들의
위치를 뒤바꾸며, 이를 p × l번 반복 (단, l은 문장의 단어 수를 의미
한다.)
4.   임의 삭제 (random Deletion, RD): (0, 1)의 범위에서 임의로 추출된
값 r이 확률 p보다 작을 경우 해당 위치에 존재하는 단어를 제거






In [ ]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!pip install git+https://github.com/ssut/py-hanspell.git

!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!git clone https://github.com/kocohub/korean-hate-speech
!git clone https://github.com/songys/Toxic_comment_data


import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from transformers import ElectraForMaskedLM
import random
import numpy as np
from hanspell import spell_checker

from tqdm.notebook import tqdm
from tqdm import tqdm

# GPU 사용
device = torch.device("cuda")

# colab에서 selenium을 돌리기 위한 옵션들
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# 형태소 분석기를 통한 품사 태깅

okt = Okt()
stem_stc = okt.pos('조사를 뭐해 와이 빵 먹자!! 23개가 ', stem=True) # 테스트 문장
stem_stc

[('조사', 'Noun'),
 ('를', 'Josa'),
 ('뭐', 'Noun'),
 ('하다', 'Verb'),
 ('와이', 'Modifier'),
 ('빵', 'Noun'),
 ('먹다', 'Verb'),
 ('!!', 'Punctuation'),
 ('23', 'Number'),
 ('개', 'Noun'),
 ('가', 'Josa')]

In [ ]:
#######################################3################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################

def remove_num(string):
    table = str.maketrans('', '', digits)
    newstring = string.translate(table)
    return newstring

def get_synonyms(keyword):
    driver = webdriver.Chrome("chromedriver", options=chrome_options)
    url_base = 'https://ko.dict.naver.com/'
    url = "https://ko.dict.naver.com/#/search?query="+keyword+"&range=word"

    driver.get(url)
    time.sleep(0.5)

    html = driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    if soup.select('a.link')[0]['href'].split("/")[1] == 'entry':
        link = soup.select('a.link')[0]['href']
    else:
        link = soup.select('a.link')[1]['href']


    driver.get(url_base + link)
    time.sleep(0.5)

    html1 = driver.page_source
    soup1 = BeautifulSoup(html1, 'html.parser')
    tags= soup1.select("div[class*=synonym] > em > a.word._word")
    synonyms = [remove_num(tag.text) for tag in tags]

    return synonyms

def synonym_replacement(sentence, n=n, seed=1):

    random.seed(seed)
    num_replaced=0
    stem_stc = okt.pos(sentence, stem=True)
    new_stem = [k[0] for k in stem_stc].copy()
    words_idx = [i for i, word in enumerate(stem_stc) if word[1] not in ['KoreanParticle', 'Josa', "Alpha", "Punctuation", "Number"]]
    josa_idx = [i for i, word in enumerate(stem_stc) if word[1] in ['Josa']] # 조사가 떨어지지 않도록
    random.shuffle(words_idx)

    if n<1:
        num_words = len(words_idx)
        n = max(1, int(n*num_words))


    for random_word_idx in words_idx:
        try:
            synonyms = get_synonyms(stem_stc[random_word_idx][0])
            if len(synonyms) >= 1:
                synonym = random.choice(synonyms)
                new_stem[random_word_idx] = synonym
                num_replaced += 1

        except:
            pass

        if num_replaced >= n: #only replace up to n words
            break

    # 동의어로 대체된 단어
    new_words = new_stem.copy()
    for josa_id in josa_idx:
        new_words[josa_id - 1] += new_stem[josa_id]
        new_words[josa_id] = ""
    return new_words

get_synonyms("많다")

['무수하다', '상당하다', '수다하다', '수없다', '숱하다', '허다하다', '무진장하다', '어마어마하다', '풍족하다']

In [ ]:
sentence = "교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정 ㅋㅋㅋ"
n=3
synonym_replacement(sentence)

['교도소', '대화', '구먼', '..', '참되다', '재미는', '', '허무하다', '..', '평점', '조정', 'ㅋㅋㅋ']

In [ ]:
########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(sentence, n=n, seed=1):
    random.seed(seed)
    stem_stc = okt.pos(sentence, stem=True)
    new_stem = [k[0] for k in stem_stc].copy()
    words_list = [i for i, word in enumerate(stem_stc) if word[1] not in ['KoreanParticle', 'Josa', "Alpha", "Punctuation", "Number"]] #remove 조사,
    random.shuffle(words_list)
    num_replaced = 0

    if n<1:
        num_words = len(words_list)
        n = max(1, int(n*num_words))

    for random_word in words_list:
        try:
            synonyms = get_synonyms(stem_stc[random_word][0])
            if len(synonyms) >= 1:
                synonym = random.choice(synonyms)
                random_idx = random.randint(0, len(new_stem)-1)
                new_stem.insert(random_idx, synonym)
                num_replaced += 1

        except:
            pass

        if num_replaced >= n: #only replace up to n words
            break

    return new_stem


In [ ]:
random_insertion(sentence)

['교도소',
 '조절',
 '이야기',
 '구먼',
 '..',
 '솔직하다',
 '재미',
 '는',
 '없다',
 '..',
 '평점',
 '참되다',
 '조정',
 '공허하다',
 'ㅋㅋㅋ']

In [ ]:

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################


def random_deletion(sentence, p, seed =1):
    random.seed(seed)
    stem_stc = okt.pos(sentence, stem=True)
    words = [k[0] for k in stem_stc].copy()

    if len(words) == 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for j, word in enumerate(stem_stc):
        if word[1] not in ['KoreanParticle', 'Josa', "Alpha", "Punctuation", "Number"]:
            r = random.uniform(0, 1)
            if r > p:
                new_words.append(word[0])
        else:
            if word[1] in ['Josa']:
                if j != 0 :
                    new_words[-1] = new_words[-1] + word[0]
                else:
                    new_words.append(word[0])
            else:
                new_words.append(word[0])



    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

In [ ]:
random_deletion(sentence, p = 0.8)

['이야기', '..는', '..', 'ㅋㅋㅋ']

In [ ]:
########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def get_words(stem_stc):
    words = []
    stem_stc_mod = stem_stc.copy()
    for j, k in enumerate(stem_stc):
        if k[1] in ['Josa']:
            if j != 0 :
                words[-1] = words[-1] + k[0]
                stem_stc_mod.remove(k)
            else:
                words.append(k[0])
        else:
            words.append(k[0])
    return words, stem_stc_mod

def random_swap(sentence, n=n, seed =1):
    random.seed(seed)
    stem_stc = okt.pos(sentence, stem = True)
    words, stem_stc_mod = get_words(stem_stc)

    words_idx = [i for i, word in enumerate(stem_stc_mod) if word[1] not in ['KoreanParticle', "Alpha", "Punctuation", "Number"]] #remove 조사,

    if n<1:
        num_words = len(words)
        n = max(1, int(n*num_words))

    if len(words_idx) == 0:
        return words

    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words, words_idx)
    return new_words



def swap_word(new_words, words_idx):
	random_idx_1 = random.choice(words_idx)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.choice(words_idx)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

In [ ]:
random_swap(sentence, n=0.1)

['교도소', '구먼', '이야기', '..', '솔직하다', '재미는', '없다', '..', '평점', '조정', 'ㅋㅋㅋ']

In [ ]:
########################################################################
# main data augmentation function
########################################################################

def eda(sentence, n_sr=0, n_ri=0, n_rs=0, p_rd=0, num_aug=9):

    augmented_sentences = []
    if_sr = n_sr > 0
    if_ri = n_ri > 0
    if_rs = n_rs > 0
    if_rd = p_rd > 0
    n_type_aug = if_sr + if_ri + if_rs + if_rd

    # if
    num_new_per_technique = math.ceil(num_aug/n_type_aug)

    #sr
    if (if_sr):
        for i in range(num_new_per_technique):
            a_words = synonym_replacement(sentence, n = n_sr, seed = i)
            augmented_sentences.append(' '.join(a_words))


    #ri
    if (if_ri):

        for i in range(num_new_per_technique):
            a_words = random_insertion(sentence, n = n_ri, seed = i)
            augmented_sentences.append(' '.join(a_words))

    #rs
    if (if_rs):
        for i in range(num_new_per_technique):
            a_words = random_swap(sentence, n = n_rs, seed = i)
            augmented_sentences.append(' '.join(a_words))

    #rd
    if (if_rd):
        for i in range(num_new_per_technique):
            a_words = random_deletion(sentence, p = p_rd, seed = i)
            augmented_sentences.append(' '.join(a_words))

    # random.shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

In [ ]:
result = eda(sentence, n_sr = 0.1)
result

['교도소 이야기 구먼 .. 솔직하다 즐거움는  없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 참되다 재미는  없다 .. 평점 조정',
 '교도소 옛날이야기 구먼 .. 솔직하다 재미는  없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미는  없다 .. 평점 조절',
 '교도소 이야기 구먼 .. 솔직하다 낙는  없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 정직하다 재미는  없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미는  없다 .. 평점 조절',
 '교도소 이야기 구먼 .. 솔직하다 재미는  없다 .. 평점 조절',
 '감방 이야기 구먼 .. 솔직하다 재미는  없다 .. 평점 조정',
 '교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정']

In [ ]:
result = eda(sentence, n_rs = 0.1)
result

['교도소 이야기 구먼 .. 없다 재미는 솔직하다 .. 평점 조정',
 '교도소 이야기 없다 .. 솔직하다 재미는 구먼 .. 평점 조정',
 '교도소 이야기 솔직하다 .. 구먼 재미는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 재미는 솔직하다 없다 .. 평점 조정',
 '평점 이야기 구먼 .. 솔직하다 재미는 없다 .. 교도소 조정',
 '교도소 이야기 구먼 .. 솔직하다 없다 재미는 .. 평점 조정',
 '교도소 구먼 이야기 .. 솔직하다 재미는 없다 .. 평점 조정',
 '이야기 교도소 구먼 .. 솔직하다 재미는 없다 .. 평점 조정',
 '교도소 조정 구먼 .. 솔직하다 재미는 없다 .. 평점 이야기',
 '교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정']

In [ ]:

result = eda(sentence, n_ri = 0.1)
result

['사담 교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 참되다 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 평점 참되다 조정',
 '교도소 이야기 구먼 .. 솔직하다 조절 재미 는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 즐거움 솔직하다 재미 는 없다 .. 평점 조정',
 '정직하다 교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 조절 평점 조정',
 '교도소 이야기 불가능하다 구먼 .. 솔직하다 재미 는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미 는 없다 .. 감방 평점 조정',
 '교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정']

In [ ]:
result = eda(sentence, p_rd = 0.1)
result

['교도소 이야기 구먼 .. 솔직하다 재미는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다는 없다 .. 평점 조정',
 '교도소 이야기 .. 재미는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미는 없다 .. 조정',
 '교도소 이야기 구먼 .. 솔직하다는 없다 .. 평점 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미는 .. 조정',
 '교도소 이야기 구먼 .. 재미는 없다 .. 조정',
 '교도소 이야기 구먼 .. 솔직하다 재미는 없다 .. 평점 조정',
 '교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정']

In [ ]:
result = eda("이영화 진짜재밋다", p_rd = 0.1, n_sr = 0.1, n_rs = 0.1, n_ri = 0.1, num_aug = 4)
result

['이영화 실지 재 밋다', '이영화 진짜 재 실지 밋다', '밋다 진짜 재 이영화', '이영화 진짜 재 밋다', '이영화 진짜재밋다']